In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display

from IPython.display import Audio
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, Conv1D, MaxPooling1D, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from scikeras.wrappers import KerasClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from scipy.stats import mode

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
dataset = pd.read_csv("dataset/features_3_sec.csv")
dataset = dataset.drop(labels="filename", axis=1)

# Get classes and encode these to numerical values (0 to 9)
class_list = dataset.iloc[:, -1]
convertor = LabelEncoder()
y = convertor.fit_transform(class_list)

# Get all data except label and standardize features
fit = StandardScaler()
X = fit.fit_transform(np.array(dataset.iloc[:, :-1], dtype = float))

In [4]:
# Split data into train, validation and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
model_loaded_xgboost = XGBClassifier()
model_loaded_xgboost.load_model("checkpoints/trained_xgb_3_sec_hw.json")
y_pred_xgboost = model_loaded_xgboost.predict(X_test)
test_accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
print(f'Testing accuracy: {test_accuracy_xgboost:.4f}')

Testing accuracy: 0.9019


In [6]:
loaded_model_fnn = load_model("checkpoints/trained_fnn_3_sec_hw.h5")
y_pred_fnn = loaded_model_fnn.predict(X_test)
y_pred_fnn_classes = np.argmax(y_pred_fnn, axis=1)
test_accuracy_fnn = accuracy_score(y_test, y_pred_fnn_classes)
print(f"Test Accuracy: {test_accuracy_fnn:.4f}")

I0000 00:00:1736284181.599798     850 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21770 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1736284182.455524    1234 service.cc:148] XLA service 0x7f8ddc0027d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736284182.455749    1234 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-01-07 22:09:42.467466: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1736284182.498645    1234 cuda_dnn.cc:529] Loaded cuDNN version 90600


51/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

I0000 00:00:1736284184.044103    1234 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step
Test Accuracy: 0.9273


In [7]:
loaded_model_cnn_1d = load_model("checkpoints/trained_cnn_1d_3_sec_hw.h5")
y_pred_cnn_1d = loaded_model_cnn_1d.predict(X_test)
y_pred_cnn_1d_classes = np.argmax(y_pred_cnn_1d, axis=1)
test_accuracy_cnn_1d = accuracy_score(y_test, y_pred_cnn_1d_classes)
print(f"Test Accuracy: {test_accuracy_cnn_1d:.4f}")

2025-01-07 22:09:48.792591: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_88', 4 bytes spill stores, 4 bytes spill loads



47/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   

2025-01-07 22:09:49.501569: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_88', 4 bytes spill stores, 4 bytes spill loads



94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Test Accuracy: 0.9042


In [16]:
y_pred_cnn_1d.shape

(2997, 10)

In [23]:
predictions = np.array([y_pred_xgboost, y_pred_fnn_classes, y_pred_cnn_1d_classes])
ensemble_predictions = mode(predictions, axis=0).mode

In [24]:
ensemble_predictions

array([4, 5, 0, ..., 6, 3, 0])

In [26]:
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Test Accuracy (Majority Voting): {ensemble_accuracy:.4f}")

Ensemble Test Accuracy (Majority Voting): 0.9329
